In [ ]:
import os
import json
import pandas as pd
from datetime import datetime
from dateutil import parser
import re
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

def parse_date_flexibly(date_str):
    if pd.isna(date_str) or not str(date_str).strip():
        return pd.NaT
    try:
        return parser.parse(date_str, fuzzy=True, default=pd.Timestamp("1900-01-01"))
    except Exception:
        return pd.NaT

def extract_year_if_possible(date_str):
    match = re.search(r"(19|20)\d{2}", str(date_str))
    if match:
        return int(match.group(0))
    return np.nan

json_dir = "/content/drive/MyDrive/dreams" # Adjust this path as needed

dreams = []
for filename in os.listdir(json_dir):
    if filename.endswith(".json"):
        with open(os.path.join(json_dir, filename), "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                dreamer = data.get("dreamer", "unknown")
                description = data.get("description", "")
                for entry in data.get("dreams", []):
                    dreams.append({
                        "dream_id": f"{dreamer}_{entry.get('number')}",
                        "dreamer": dreamer,
                        "number": entry.get("number"),
                        "description": description,
                        "date_raw": entry.get("head"),
                        "content": entry.get("content")
                    })
            except Exception as e:
                print(f"Error parsing {filename}: {e}")

df = pd.DataFrame(dreams)
df["text_clean"] = df["content"].str.replace(r"\s+", " ", regex=True).str.strip()
df["date_parsed"] = df["date_raw"].apply(parse_date_flexibly)
df["year"] = df["date_raw"].apply(extract_year_if_possible)

# Save
os.makedirs("data/processed", exist_ok=True)
df.to_csv("data/processed/cleaned_dreams.csv", index=False)

df.head()


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/raw/'